In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.summit.utils.plotting import plot
from lsst.obs.lsst import LsstCam
from lsst.meas.algorithms.maskStreaks import Line, LineProfile, LineCollection, MaskStreaksTask
from scipy.ndimage import map_coordinates

In [ ]:
#butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
#                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])
camera = LsstCam.getCamera()
instrument = "LSSTCam"
streakTask = MaskStreaksTask()

In [ ]:
def line_profile(img, line, W, L, nW=200, nL=200):
    """
    Compute a profile perpendicular to a line defined by (rho, theta),
    where theta is the angle from vertical, and rho is the perpendicular distance
    from the image center to the line.

    The profile runs perpendicular to the line (width W)
    and is averaged along the line (length L).
    """
    rho = line.rho
    theta = line.theta
    theta *= -np.pi / 180.0 # convert to radians
    ny, nx = img.shape
    xc, yc = nx / 2, ny / 2

    # direction vectors
    v = np.array([np.sin(theta), np.cos(theta)])   # along the line
    n = np.array([np.cos(theta), -np.sin(theta)])  # normal (perpendicular) to the line

    # compute point on the line closest to the image center
    x0 = xc + rho * n[0]
    y0 = yc + rho * n[1]

    # coordinates along perpendicular (profile axis) and along line (averaging direction)
    s = np.linspace(-W/2, W/2, nW)
    t = np.linspace(-L/2, L/2, nL)
    S, T = np.meshgrid(s, t)

    # map (S, T) to image coordinates
    X = x0 + T * v[0] + S * n[0]
    Y = y0 + T * v[1] + S * n[1]

    # interpolate values from image
    values = map_coordinates(img, [Y, X], order=1, mode='nearest')

    # average along the line direction
    profile = np.median(values,axis=0)

    return s, profile, values

# First run one CCD as a test

In [ ]:
# Faint
#dayObs = 20250915
#seqNum = 318
#detNum = 55
# Even fainter
#dayObs = 20250915
#seqNum = 319
#detNum = -1
#detName = 'R20_S22'

# Brighter
dayObs = 20250909
seqNum = 313
detNum = 106

expId = int(dayObs * 1E5 + seqNum)
# Can specify detNum or detName
if detNum < 0:
    det = camera[detName]
    detNum = det.getId()
print(detNum)
calexp = butler.get('preliminary_visit_image', detector=detNum, visit=expId, instrument=instrument)


In [ ]:
import pickle as pkl
filename = "/home/c/cslage/u/Satellites/streak_images/calexps.pkl"
with open(filename, 'rb') as f:
    [calexp_20250915_319_80, calexp_20250915_318_55, calexp_20250909_313_106] = pkl.load(f)

In [ ]:
# Faint
#dayObs = 20250915
#seqNum = 318
#detNum = 55
# Even fainter
#dayObs = 20250915
#seqNum = 319
#detNum = 80

# Brighter
dayObs = 20250909
seqNum = 313
detNum = 106

calexp = eval(f"calexp_{dayObs}_{seqNum}_{detNum}")

In [ ]:
plt.imshow?

In [ ]:
import matplotlib.colors as mcolors
images =[[20250909, 313, 106], [20250915, 318, 55], [20250915, 319, 80]]
titles = ['SNR ~ 10', 'SNR ~ 1', 'SNR < 1']
fig, axs = plt.subplots(1, 3, figsize = (20, 10))
#norm_log = mcolors.LogNorm(vmin=0.1, vmax=1000)
plt.subplots_adjust(wspace=0.01)
calexps = []
for i, [dayObs, seqNum, detNum] in enumerate(images):
    calexp = eval(f"calexp_{dayObs}_{seqNum}_{detNum}")
    arr = calexp.image.array
    arr -= np.nanmedian(arr)
    calexps.append(arr)
for i, ax in enumerate(axs):
    im = ax.imshow(calexps[i], cmap='gray', vmin=0, vmax=20)
    ax.set_title(titles[i])
    ax.axis('off') # Optional: turn off axes for a cleaner image display
cbar = fig.colorbar(im, ax=axs.ravel().tolist(), orientation='vertical', anchor=(-0.3, 0.5), shrink=0.50)
cbar.set_label('e-/pixel surface brightness', rotation=90, labelpad=10, fontsize=12) 

plt.savefig("/home/c/cslage/u/Satellites/streak_images/Combined_streaks.png")


In [ ]:
%matplotlib inline
x = plot(calexp, stretch='linear')#, percentile=5)
x.savefig(f"/home/c/cslage/u/Satellites/streak_images/Image_{dayObs}_{seqNum}_{detNum}.png")
x

# Now we test the code with the different output options

In [ ]:
struct = streakTask.run(calexp)
lines = struct.lines
for line in lines:
    print(line)

In [ ]:
arr = calexp.image.array
weights = np.ones_like(arr, dtype=bool)
#line = lines[0]
line = Line(800, 71) # Telling it where to look
line.sigma = 2.0
lineModel = LineProfile(arr, weights, line=line)

In [ ]:
plt.imshow(lineModel.lineMask, origin='lower')

In [ ]:
fit, fitFailure = lineModel.fit()
print(fit.rho, fit.theta)
finalModel = lineModel.makeProfile(fit)
plt.imshow(finalModel, origin='lower', cmap='gray', vmin=0, vmax=10)
plt.title(f"Final Model {dayObs} {seqNum} {detNum}")
plt.savefig(f"/home/c/cslage/u/Satellites/streak_images/Final_Model_{dayObs}_{seqNum}_{detNum}.png")

In [ ]:
W = 100
L = 2000
nW = 100
nL = 2000

s, profile, values = line_profile(calexp.image.array, line, W, L, nW=nW, nL=nL)
s_model, profile_model, values_model = line_profile(finalModel, line, W, L, nW=nW, nL=nL)
s_mask, profile_mask, values_mask = line_profile(lineModel.lineMask, line, W, L, nW=nW, nL=nL)
plt.title(f"Streak profile {dayObs} {seqNum} {detNum}")
plt.plot(profile, label="Measured")
plt.plot(profile_model, label="Fit")
plt.plot(profile_mask * np.max(profile), ls='--', label="Mask")
plt.xlabel("X (pixels)")
plt.ylabel("Flux (electrons)")
plt.legend()
plt.savefig(f"/home/c/cslage/u/Satellites/streak_images/Line_Profile_{dayObs}_{seqNum}_{detNum}.png")

# Sky background is ~2500, sqrtof that is 50, so this is a SNR ~ 1??

In [ ]:
W = 100
L = 2000
nW = 100
nL = 2000

s, profile, values = line_profile(calexp.image.array, line, W, L, nW=nW, nL=nL)
s_model, profile_model, values_model = line_profile(finalModel, line, W, L, nW=nW, nL=nL)
s_mask, profile_mask, values_mask = line_profile(lineModel.lineMask, line, W, L, nW=nW, nL=nL)
plt.title(f"Streak profile {dayObs} {seqNum} {detNum}")
plt.plot(profile, label="Measured")
plt.plot(profile_model, label="Fit")
plt.plot(profile_mask * np.max(profile), ls='--', label="Mask")
plt.xlabel("X (pixels)")
plt.ylabel("Flux (electrons)")
plt.legend()
plt.ylim(-10,20)
plt.savefig(f"/home/c/cslage/u/Satellites/streak_images/Line_Profile_BlowUp_{dayObs}_{seqNum}_{detNum}.png")

# Sky background is ~2500, sqrtof that is 50, so this is a SNR ~ 1??

In [ ]:
plt.imshow(values, origin='lower', cmap='gray', vmin=0, vmax=10)

In [ ]:
W = 400
L = 1
nW = 400
nL = 1

s, profile, values = line_profile(calexp.image.array, line, W, L, nW=nW, nL=nL)
s_model, profile_model, values_model = line_profile(finalModel, line, W, L, nW=nW, nL=nL)
plt.title(f"Streak profile {dayObs} {seqNum} {detNum}")
plt.plot(profile, label="Measured")
plt.plot(profile_model, label="Fit")
plt.xlabel("X (pixels)")
plt.ylabel("Flux (electrons)")
plt.legend()
plt.savefig(f"/home/c/cslage/u/Satellites/streak_images/Line_Profile_1_{dayObs}_{seqNum}_{detNum}.png")
